In [2]:
import deepl
import pandas as pd
import numpy as np

In [3]:
auth_key = "XXXXXXXXXXXXXXX"  # Replace with your key
translator = deepl.Translator(auth_key)


In [102]:
usage = translator.get_usage()
if usage.any_limit_reached:
    print('Translation limit reached.')
if usage.character.valid:
    print(
        f"Character usage: {usage.character.count} of {usage.character.limit}")
if usage.document.valid:
    print(f"Document usage: {usage.document.count} of {usage.document.limit}")

Character usage: 4248368 of 10000000


## Import data

In [8]:
path = '/Users/jinqiaoli/Desktop/Thesis/data/taskontology/'
origin = 'onet_occ_tasks_workactivities_en.jsonl'
sjmm_de = 'onet_occ_tasks_workactivities_en2de.jsonl'

origi_df = pd.read_json(path+origin,  lines=True)
print(len(origi_df))
print('unique task: ',len(origi_df['text'].value_counts()))
origi_df.head()

22210
unique task:  20884


,id,type,language,text
0,11-1011.00,SOCCode,en,Chief Executives
1,20461,Task,en,"Review and analyze legislation, laws, or publi..."
2,8823,Task,en,Direct or coordinate an organization's financi...
3,8824,Task,en,"Confer with board members, organization offici..."
4,8825,Task,en,Analyze operations to evaluate performance of ...


In [9]:
# german data translated with SJMM
sjmm_de = pd.read_json(path+sjmm_de, lines=True)
print(len(sjmm_de))
print('unique task: ',len(sjmm_de['text'].value_counts()))
sjmm_de.head()

22210
unique task:  20875


,id,type,language_orig,language,text
0,11-1011.00,SOCCode,en,de,Chief executives
1,20461,Task,en,de,Überprüfung und Analyse von Rechtsvorschriften...
2,8823,Task,en,de,Direkt oder koordiniert eine Organisation fina...
3,8824,Task,en,de,"Controlling mit Vorstandsmitgliedern, Organisa..."
4,8825,Task,en,de,"Sie analysieren Operationen, um die Leistung e..."


## Sample test

In [103]:
# sample test
result = translator.translate_text("How are you", target_lang="DE", formality="less")
print(result.text)  # Wie geht es dir
result = translator.translate_text("How are you", target_lang="DE", formality="more")
print(result.text) # Wie sind Sie


## split test
ori_part1 = origi_df['text'][:1000].tolist()
print(ori_part1)
results = translator.translate_text(ori_part1, target_lang="DE")
print(len(results))
results_text = []
for i in results:
    results_text.append(i.text)
len(results_text)

Wie geht es dir
Wie sind Sie


# Main tranlation part

In [93]:
## split into small parts for translation
parts = len(origi_df['text'])//1000 # 22
deepl_text = []


# translate with DeepL API
for i in range (parts+1):
    part = origi_df['text'][1000*i:1000*(i+1)]
    results = translator.translate_text(part, target_lang="DE")
    for re in results:
        deepl_text.append(re.text)

In [98]:
# creat a new dataframe
deepl_de = sjmm_de[['id','type','language_orig','language']]
deepl_de['text'] = deepl_text

# save to .json file
outfile = 'onet_occ_tasks_workactivities_en2de_deepl.jsonl'
print('saved translated file to: ', path+outfile)
deepl_de.to_json(path+outfile)

deepl_de

,id,type,language_orig,language,text
0,11-1011.00,SOCCode,en,de,Hauptgeschäftsführer
1,20461,Task,en,de,Überprüfung und Analyse von Rechtsvorschriften...
2,8823,Task,en,de,Leiten oder koordinieren Sie die Finanz- oder ...
3,8824,Task,en,de,"Besprechung mit Vorstandsmitgliedern, Organisa..."
4,8825,Task,en,de,"Analysieren Sie Vorgänge, um die Leistung eine..."
...,...,...,...,...,...
22205,4.A.4.c.3.I07.D01,DWA,en,de,Überwachen Sie die Verfügbarkeit von Ausrüstun...
22206,4.A.4.c.3.I07.D02,DWA,en,de,Inventarisierung von Materialien oder Ausrüstung.
22207,4.A.4.c.3.I07.D03,DWA,en,de,Inventarisierung von medizinischem Material od...
22208,4.A.4.c.3.I07.D04,DWA,en,de,Überwachung der Bestände an Produkten oder Mat...


## read saved json file

In [100]:
deepl_df = pd.read_json(path+outfile)
deepl_df

,id,type,language_orig,language,text
0,11-1011.00,SOCCode,en,de,Hauptgeschäftsführer
1,20461,Task,en,de,Überprüfung und Analyse von Rechtsvorschriften...
2,8823,Task,en,de,Leiten oder koordinieren Sie die Finanz- oder ...
3,8824,Task,en,de,"Besprechung mit Vorstandsmitgliedern, Organisa..."
4,8825,Task,en,de,"Analysieren Sie Vorgänge, um die Leistung eine..."
...,...,...,...,...,...
22205,4.A.4.c.3.I07.D01,DWA,en,de,Überwachen Sie die Verfügbarkeit von Ausrüstun...
22206,4.A.4.c.3.I07.D02,DWA,en,de,Inventarisierung von Materialien oder Ausrüstung.
22207,4.A.4.c.3.I07.D03,DWA,en,de,Inventarisierung von medizinischem Material od...
22208,4.A.4.c.3.I07.D04,DWA,en,de,Überwachung der Bestände an Produkten oder Mat...
